In [29]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from time import sleep

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36')
options.add_argument('window-size=1920,1080')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 크롬 드라이버 설정
#service = Service(executable_path='chromedriver')  # chromedriver 경로 지정
driver = webdriver.Chrome(options=options)

# 대기 시간 설정
driver.implicitly_wait(5)

# 데이터를 저장할 리스트
blog_data = []

# 블로그 정보를 크롤링하는 함수
def get_blog_info(page_number, keyword):
    try:
        # 네이버 블로그 검색 페이지 열기
        URL = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_number}&rangeType=ALL&orderBy=sim&keyword={keyword}'
        driver.get(URL)
        sleep(2)  # 페이지 로딩 대기

        # 블로그 글 리스트 수집
        try:
            posts = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, 'div.desc > a.desc_inner')
            ))
        except TimeoutException:
            print(f"{keyword}: 검색 결과가 없습니다.")
            return

        print(f"{keyword}: 현재 {page_number} 페이지 / 총 {len(posts)}개의 블로그 글을 찾았습니다.\n")

        for index, post in enumerate(posts, start=1):
            try:
                # 블로그 글 제목
                post_title = post.find_element(By.CSS_SELECTOR, 'strong').text
                print(f"{index}. 제목: {post_title}")

                # 글 링크를 열어 상세 정보 수집
                post_url = post.get_attribute('href')
                driver.get(post_url)
                sleep(2)

                # iframe 전환
                try:
                    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, 'iframe#mainFrame')))
                    print("iframe으로 전환되었습니다.")
                except TimeoutException:
                    print("iframe 전환 실패. 데이터를 수집할 수 없습니다.")
                    continue

                # 게시자
                try:
                    author = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@class, 'nick')]"))).text
                except NoSuchElementException:
                    author = '정보 없음'

                # 게시 날짜
                try:
                    date = driver.find_element(By.XPATH, "//span[contains(@class, 'se_publishDate') or contains(@class, 'post_date')]").text
                except NoSuchElementException:
                    date = '정보 없음'

                # 블로그 내용
                try:
                    content = driver.find_element(By.XPATH, "//div[contains(@class, 'se-main-container') or contains(@class, 'post-view')]").text
                except NoSuchElementException:
                    content = '정보 없음'

                # 수집한 데이터를 리스트에 저장
                blog_data.append({
                    '제목': post_title,
                    '게시자': author,
                    '게시 날짜': date,
                    '내용': content,
                    'URL': post_url
                })
                print(f"수집된 데이터: {blog_data[-1]}\n")

                # 이전 검색 페이지로 돌아가기
                driver.back()
                sleep(2)

            except Exception as e:
                print(f"{index}번 블로그 정보 수집 중 오류 발생: {e}")
                continue

    except Exception as e:
        print(f"검색 중 오류 발생: {e}")

# 검색어와 페이지 설정
keyword = '석촌 전통시장'
total_pages = 2  # 원하는 페이지 수 설정

# 각 페이지에 대해 크롤링 수행
for page in range(1, total_pages + 1):
    get_blog_info(page, keyword)

# 크롤링 완료 후 브라우저 닫기
driver.quit()

# Pandas DataFrame으로 변환
df = pd.DataFrame(blog_data)
#print(df)


석촌 전통시장: 현재 1 페이지 / 총 7개의 블로그 글을 찾았습니다.

1. 제목: 석촌 전통시장 맛집 [코리안바베큐] 숯불 닭구이 치킨집!?
iframe으로 전환되었습니다.
수집된 데이터: {'제목': '석촌 전통시장 맛집 [코리안바베큐] 숯불 닭구이 치킨집!?', '게시자': '', '게시 날짜': '2024. 7. 19. 9:20', '내용': '송파역 맛집 석촌역 맛집 석촌 전통시장 숯불 닭구이 전문점\n안녕하세요!\n맛집과 여행을 사랑하는 허PD입니다\n오늘은 석촌역 맛집이자 송파역 맛집이라고도 불리는\n석촌호수 바로 앞에 위치한 석촌 전통시장 맛집을 다녀왔어요!\n석촌 전통시장 초입부분에 위치한 코리안 바베큐라는 이곳\n굉~장히 유명합니다\n내부도 넓고 이른시각인데 사람들이 있었다...\n밖에서 보는 너무 맛있어 보인다...\n엄청 열심히 굽고 계시는 사장님...!ㅋㅋㅋ\n이걸보고 안들어올수없다!!!\n참고로, 여기 정말 석촌 전통시장 맛집들중에서 굉장히 유명한 곳이라고 해요\n일단, 주방 위생상태는 그냥 그럭저럭...? 평범했어요 솔직히 깔끔한건 아니였다\n너무 솔직한가 나...\n근데 여기 맛집으로 소문난게 유명해...! 닭구이가 끝장나거든\n내부는 4~6인 테이블이 대략 15개~20개는 있었다\n즉, 단체석도 가능!\n넓기도 넓었고 우드톤으로 나무디자인의 테이블들이였어요\n그리고 창가도 있었다\n기본반찬으로는 더 샐러드!\n엄청나게 크게 많이 나오죠? 좀 당황스러웠었어요ㅋㅋㅋ\n큰접시에 저렇게 담아서 주다니...\n오히려 좋아!\n메뉴판을 보시면 아시겠지만\n생각보다 너무 잘되어있습니다\n세트메뉴는 없지만 맛있고 잘 나가는 메뉴들이 있는데 제가 알려드릴게요\n(같이 온 지인이 여기 석촌호수맛집 전문가임)\n그리고 나온 한식바베큐와 소금구이!\n매운거 하나시키고 안매운거 하나 시켰어요ㅎㅎ\n소금구이가 안매운것 중에서 잘 나간다고 해요!\n제가 먹어봤을때\n맛은 있는데 가격에 비해서 양이나 맛이 크~게 미친듯이 맛있진 않았다\n그냥 치킨집인데 나